# Notebook 07: Acetogenic Interaction Model

This notebook is part of the research paper titled:

Optimizing acetogenic CO(2) utilization through metabolic and process engineering highlights the potential of thermophilic bioproduction

This notebook uses a stoichiometric constraint-based model of the Wood-Ljungdahl pathway jointly with a genome-scale model of Paragoebacillus thermoglucosidasius, to model a joint cultivation of both organisms symbiotically fermenting syngas to produce acetone, through acetate as an exchange metabolite.

Notebook author: Philip J. Gorter de Vries

In [1]:
import numpy as np
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite

# Load Acetogen Model and Predict Theoretical Yields

In [2]:
WLModel = cobra.io.read_sbml_model('../Models/WLPathway_bifurcating.xml')

medium_carb = WLModel.medium
medium_carb['EX_co_e'] = 1000
medium_carb['EX_co2_e'] = 0
medium_carb['EX_h2_e'] = 0

medium_hom = WLModel.medium
medium_hom['EX_co_e'] = 0
medium_hom['EX_co2_e'] = 1000
medium_hom['EX_h2_e'] = 1000


--------------------------------------------
--------------------------------------------

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [3]:
WLModel.objective = "EX_ac_e"

with WLModel:
    WLModel.medium = medium_carb
    print("Carboxydotrophic acetate production flux: ", WLModel.slim_optimize(), "\n")
    Acetate_ex_carb = WLModel.slim_optimize()
    CO2_exc_carb = WLModel.reactions.EX_co2_e.flux/Acetate_ex_carb
    H2_exc_carb = WLModel.reactions.EX_h2_e.flux/Acetate_ex_carb
    CO_exc_carb = WLModel.reactions.EX_co_e.flux/Acetate_ex_carb
    H2O_exc_carb = WLModel.reactions.EX_h2o_e.flux/Acetate_ex_carb

with WLModel:
    WLModel.medium = medium_hom
    print("Homoacetogenic acetate production flux: ", WLModel.slim_optimize(), "\n")
    Acetate_ex_hom = WLModel.slim_optimize()
    CO2_exc_hom = WLModel.reactions.EX_co2_e.flux/Acetate_ex_carb
    H2_exc_hom = WLModel.reactions.EX_h2_e.flux/Acetate_ex_carb
    CO_exc_hom = WLModel.reactions.EX_co_e.flux/Acetate_ex_carb
    H2O_exc_hom = WLModel.reactions.EX_h2o_e.flux/Acetate_ex_carb
    
print(f"Carboxydotroph: {-CO_exc_carb} CO + {round(-H2O_exc_carb,2)} H2O => {CO2_exc_carb} CO2 + {1} acetate")
print(f"Homoacetogen: {-CO2_exc_hom} CO2 + {-H2_exc_hom} H2 => {round(H2O_exc_hom,2)} H2O + {1} acetate")

Y_IE_carb = -1/CO_exc_carb*2
Y_IE_hom = -1/H2_exc_hom*2
print(f"Carboxydotrophic acetate yield: {Y_IE_carb} [Cmol/emol]")
print(f"Homoacetogenic acetate yield: {Y_IE_hom} [Cmol/emol]")

Carboxydotrophic acetate production flux:  1.0 

Homoacetogenic acetate production flux:  1.0 

Carboxydotroph: 4.0 CO + 0.33 H2O => 2.0 CO2 + 1 acetate
Homoacetogen: 2.0 CO2 + 4.0 H2 => 2.33 H2O + 1 acetate
Carboxydotrophic acetate yield: 0.5 [Cmol/emol]
Homoacetogenic acetate yield: 0.5 [Cmol/emol]


# Add Heterotroph GEM

In [4]:
#Bsubt = cobra.io.read_sbml_model(join(data_dir,'Models/iJO1366.xml'))
Geob = cobra.io.read_sbml_model('../Models/p-thermo_anaerob_actn.xml')

Geob.reactions.GAPDH.bounds = 0,1000 #Futile cycle discovered, due to error in bounds

#Fix bounds for acetate uptake
Geob.reactions.PTAr.bounds = -1000,1000
Geob.reactions.ACKr.bounds = -1000,1000

Change objective function and remove constraints for ATP uptake for maintenance. This allows to ignore both GAM and NGAM

In [5]:
medium = Geob.medium
medium["EX_glc__D_e"] = 0
medium["EX_o2_e"] = 1000
medium["EX_ac_e"] = 1
Geob.medium = medium

solution1 = cobra.flux_analysis.pfba(Geob)
#Y_PI_1 = -(solution1.fluxes.EX_act_e*3)/(solution1.fluxes.EX_ac_e*2)
Y_SO_1 = solution1.fluxes.EX_ac_e/2/solution1.fluxes.EX_o2_e

#print(f"Heterotrophic acetone yield on acetate: {round(Y_PI_1,2)} [Cmol/Cmol]")

print("aerobic acetate for growth")
print(f"Substrate to oxygen ratio: {round(Y_SO_1,2)} [Cmol/mol]")
Geob.summary()

aerobic acetate for growth
Substrate to oxygen ratio: 0.27 [Cmol/mol]


Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,1,2,100.00%
ca2_e,EX_ca2_e,1.345E-05,0,0.00%
cobalt2_e,EX_cobalt2_e,9.523E-07,0,0.00%
fe2_e,EX_fe2_e,0.0001481,0,0.00%
h_e,EX_h_e,0.9655,0,0.00%
k_e,EX_k_e,0.003024,0,0.00%
mg2_e,EX_mg2_e,0.0004185,0,0.00%
nh4_e,EX_nh4_e,0.04131,0,0.00%
o2_e,EX_o2_e,1.82,0,0.00%
pi_e,EX_pi_e,0.003703,0,0.00%


In [6]:
Builder(model = Geob, map_json = '../Models/EscherMaps/P-thermo_ReversibleAcetateMet.json', reaction_data = solution1.fluxes.to_dict())

Builder(reaction_data={'IDPh': 0.011012510828839774, 'CAT': 4.186575639938549e-06, 'PDHam1hi': 0.0, 'HYDA': 0.…

In [7]:
Geob.objective = "EX_act_e" #don't force growth

Geob.reactions.ATPM.bounds = 0,1000 #Ignore NGAM
Geob.reactions.ATPM.bounds

(0, 1000)

In [8]:
solution2 = cobra.flux_analysis.pfba(Geob)

Y_PI_2 = -(solution2.fluxes.EX_act_e*3)/(solution2.fluxes.EX_ac_e*2)
Y_SO_2 = (solution2.fluxes.EX_ac_e/2)/solution2.fluxes.EX_o2_e

print(f"Heterotrophic acetone yield on acetate: {round(Y_PI_2,2)} [Cmol/Cmol]")
print(f"Substrate to oxygen ratio: {round(Y_SO_2,2)}")

Geob.summary()

Heterotrophic acetone yield on acetate: 0.71 [Cmol/Cmol]
Substrate to oxygen ratio: 4.69


Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,1,2,100.00%
h_e,EX_h_e,1,0,0.00%
o2_e,EX_o2_e,0.1067,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
act_e,EX_act_e,-0.4733,3,71.00%
co2_e,EX_co2_e,-0.58,1,29.00%
h2o_e,EX_h2o_e,-0.58,0,0.00%


In [9]:
Builder(model = Geob, map_json = '../Models/EscherMaps/P-thermo_AcetateDegrad.json', reaction_data = solution2.fluxes.to_dict())

Builder(reaction_data={'IDPh': 0.0, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, 'PPBNGS': 0.0, …

More or less in accordance with:  10.3390/ijms21228777
Check and compare critically though

In [10]:
Builder(model = Geob, map_json = '../Models/EscherMaps/AcetonePathway.json', reaction_data = solution2.fluxes.to_dict())

Builder(reaction_data={'IDPh': 0.0, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, 'PPBNGS': 0.0, …

# Combine into Yields

In [11]:
print(f"Succesive carboxydotrophic acetone yield: {round(Y_IE_carb*Y_PI_2,2)} [Cmol/emol]")
print(f"Succesive homoacetogenic acetone yield: {round(Y_IE_hom*Y_PI_2,2)} [Cmol/emol]")

Succesive carboxydotrophic acetone yield: 0.36 [Cmol/emol]
Succesive homoacetogenic acetone yield: 0.36 [Cmol/emol]
